- Ggf. zu float16 konvertieren checken, ob finaler df mit time slices dann deutlich kleiner und performance testen
- Cross validation?
- Ensemble learning?

In [1]:
#Import data handling libraries
import pandas as pd
import numpy as np

In [2]:
#Specify directories
code_env = 'kaggle'
#code_env = 'local'


if code_env=='local':
    ###local###
    #get parent folder of current directory
    parent_dir = '/Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction'

    #Directory resources
    res_dir = parent_dir + '/res/'
    src_dir = parent_dir + '/src/'
    prc_dir = src_dir + 'processed_data/' # Processed data directory with pickled dataframes

if code_env=='kaggle':
    ###On Kaggle###
    res_dir = '/kaggle/input/m5-forecasting-accuracy/'
    prc_dir = '/kaggle/input/processed-data/'

Parent directory of current notebook: /Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction
Resource directory: /Users/mf/Desktop/CS/Studies/7_Final_Project/Kaggle_M5PointPrediction/res/


In [3]:
# Create variables
BASE     = prc_dir +'df_1.pkl'
CALENDAR = prc_dir +'df_2.pkl'

In [4]:
# Read in df_train_conv from pickle file
df_all_data = pd.concat([pd.read_pickle(BASE),
           pd.read_pickle(CALENDAR)], 
           axis=1)

In [5]:
# Splitting the data
TRAIN_END = 1913 - 28 # Train only until the 28 days before the end of the data
VAL_END = 1913

df_train = df_all_data[df_all_data['d'] <= TRAIN_END]
#df_val = df_all_data[(df_all_data['d'] > TRAIN_END) & (df_all_data['d'] <= VAL_END)]
#df_test = df_all_data[df_all_data['d'] > VAL_END]

del df_all_data

In [6]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam

In [7]:
# Define categorical and numerical columns
categorical_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wday', 
                    'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 
                    'snap_CA', 'snap_TX', 'snap_WI', 'mday', 'week', 'month', 'year']
numerical_cols = ['sell_price']

target_col = 'sales_amount'

# Convert categorical columns to category dtype and encode with cat.codes
for col in categorical_cols:
    df_train[col] = df_train[col].astype('category').cat.codes

# Normalize numerical columns
scaler = MinMaxScaler()
df_train[numerical_cols] = scaler.fit_transform(df_train[numerical_cols].astype(np.float32))
df_train['sales_amount'] = scaler.fit_transform(df_train[['sales_amount']].astype(np.float64))


# LSTM Data Preparation
time_steps = 7  # Number of time steps (can be tuned)
batch_size = 32  # Batch size for the generator


/var/folders/w1/_96k6h412s5d15f14hdtbbyc0000gn/T/ipykernel_56562/191470035.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[col] = df_train[col].astype('category').cat.codes
/var/folders/w1/_96k6h412s5d15f14hdtbbyc0000gn/T/ipykernel_56562/191470035.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[col] = df_train[col].astype('category').cat.codes
/var/folders/w1/_96k6h412s5d15f14hdtbbyc0000gn/T/ipykernel_56562/191470035.py:11: SettingWithCopyWarning: 
A value is trying to be set on 

In [8]:
# give me row number for each time the value in column d changes
#df_train.groupby('d')['d'].apply(lambda x: x.index[0]).values
#give me the code to calculate the difference between each two adjacent values in an array
#np.diff(df_train.groupby('d')['d'].apply(lambda x: x.index[0]).values)
#df_train.info()

In [9]:
# Custom RMSE loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [10]:
def create_sequences(df, target, days_per_sequence=1):    
    num_items = df_train[df_train['d'] == 1].shape[0]  # Number of items per day (day 1 examplary)
    X, y = [], []

    for i in range(0, len(df) - num_items * days_per_sequence, num_items):
        end_idx = i + num_items * days_per_sequence
        X.append(df.iloc[i:end_idx, :].drop(target, axis=1).values)
        y.append(df.iloc[end_idx:end_idx + num_items][target].values)

    return np.array(X), np.array(y)

# Usage
time_steps = 7  # Number of days per sequence
X_train, y_train = create_sequences(df_train, target_col, days_per_sequence=time_steps)


: 

In [ ]:
# Model

In [ ]:
# After prediction
# predictions_normalized = model.predict(...)  # Predictions in the normalized scale
# predictions_original = scaler.inverse_transform(predictions_normalized)  # Inverse-transformed predictions